<span type="title">泛型</span> | <span type="version">1.0</span>  | <span type="update">2018-07-29</span> 

<span type="intro"><p class="card-text">本章主要介绍Java中的泛型机制。泛型指的是通过参数化类型而达到代码复用的机制。如果在继承一个类，或者在写一个接口或者方法的时候，发现新的对象除了类型之外和旧对象过于相似，那么就可以通过泛型来复用代码。另一种情况是，如果使用已有代码时现有的类、接口过于抽像，而你又不想因此多继承、实现几个类，这时通过泛型来提高具象。本质上说，泛化就是用于这种情况及其变体下的代码复用。泛型的特征：类的指代可以使得我们可以利用它来来实现某些功能，比如元组。</p><p class="card-text">本章从泛型解决的问题开始谈起，接着使用一个元组、一个堆栈、一个随机生成器来展示了泛型常见的应用。泛型不仅可用于类，还可用于接口和方法，用作接口时常用来动态更改方法的返回类型，用作方法时可以用来进行动态填充参数和返回值、进行参数推断等，在第二部分详细介绍了可变参数、简化元组、生成器对象等在泛型方法上的应用。接下来，介绍了泛型在匿名内部类上的使用情况，介绍了使用泛型带给编程的优势：更少的类和更小的耦合。</p><p class="card-text">在第三部分，介绍了泛型的内部机制：擦除以及其带来的问题、边界处的泛型行为，提到了我们对于擦除的补偿：使用协变、类字面量和工厂以创建对象和保留对象信息。接下来讲解了通配符，提到了协变和逆变的区别，功能差异问题。最后，简要的说明了在Java上使用泛型带来的问题：基本类型、重载、基类劫持等。最后介绍了相比较其他语言的潜在类型机制，Java的一些补足办法：接口、RTTI以及适配器。</p></span>

# 泛型概述

泛型是什么？泛型是一种依附于类、类方法、接口的能够保存类型信息的一种结构，通常用`<T>`表示。在类、类方法、接口的内部，可以把T作为形参使用。当我们调用一个类、类方法或者接口的时候，可以往这个尖括号中传递一个具体的类型，这样的话，T就像导光管，在其作用域内部所有使用T的地方传递具体的参数。泛型就是这样一种导光装置：在一个作用域内连接其实参和形参。

当类、类方法、接口定义的时候，使用形参，然后在这些结构使用的时候，通过某种方法传递实参，那么实参就可以替换这些地方的形参。从这个角度看，泛型可以接受任意类型的实际参数，而把它们表示成形式参数，就好像写了无数个本身就带有实际参数的类、方法、接口一样。因此常常认为，泛型促进了代码复用。

**泛型之前的问题解决思路**

在一个类/方法中，要么只能使用基础类型，要么只能使用自定类型。这对于程序来讲是一个很大的限制。为什么呢？当我们使用自定类型的时候，我们目前在写的这个类就只能限制使用这些我们声明的类型，因此这个类的使用就非常受限，它只能够处理少数几个类的行为。

面向对象的世界里提供了几种解决这个问题的思路：

1、使用多态，这样不论是单继承还是使用内部类的多继承都可以通过动态绑定、向上转型，让我们只用访问基类/抽象类的方法就可以使用所有导出类重载的方法。但这仅限于重载的方法。

2、使用接口。接口是一种很灵活的机制，任何类都能实现一个接口，实现了接口的抽象类甚至可以将多态和接口合并起来作为接口使用。因此，当在类中调用接口定义的方法，就可以访问所有实现了该接口的方法。这也是一种泛化。

但是这两种泛化应用依旧受限，接口的解决方案需要我们要用的每个类都必须实现接口，多态的解决方案要求我们使用基类。它们都有外在的结构要求。

而，对于一个类、方法、接口的作用域，如果我们在设计的时候假设有一个为T的类型会被使用，那么在这些作用域内就可以用T来进行各种行为类型代指：表示一个变量的类型、作为某个方法的返回值、作为某个方法的参数等等。这样的话，当在构造这个类/方法/接口的时候，我们传递进去一个具体的类型，这些T自动的被替换成这个具体的类型，就像我们设计了一个专门为这个具体类型设计的类/方法/接口一样。

# 泛型基础

在下面的几部分将简要的介绍泛型以及其应用。

**泛型的定义**：泛型就是参数化的类型。当你写一个类或者接口，发现某个类名重复出现，那么就可以将其替换为形参，加上泛型声明，在使用的时候只需要传递实参即可，这样就达到了代码复用的目的，这个类和任意类组合使用相同代码。泛型还有很多额外的好处，比如，你可以在一个作用域中嵌套多个其他具有泛型的作用域，直接将形参传递进去，如果没有泛型，那么就只能创建一个具体引用，然后再传递过去。

**泛型的特性**：泛型的典型特性就是通过这样一种在作用域中使用形参代替实参的方法，构造了一根连接形参和实参的导光管。最典型的就是元组的Java实现，任意类型的类被传递到一个方法中，这个方法设计用来接收T类型参数，而这个方法体中，通过将这个T类型传递给一个接受泛型参数的类进行处理，在这个类中进行了T类型的初始化。因此，当我们将一个Int类型的参数传递给这个方法的时候，通过导光管，这个形参一直被传递到这个类的域定义的参数那里得到了初始化。


**在接下来的介绍中，泛型的介绍将分为：
1、在类、类方法、接口中使用形式变量对泛型进行构造
2、在客户端传递实际参数使用泛型
这两大部分，请勿将这两部分混淆。**


## 泛型类的设计和定义

```java
import static com.mazhangjing.Print.*;
class Holder2<T> {
    private T t;
    Holder2(T t){this.t = t;}
    T getElement(){
        return t;
    }
}
```

泛型最引人注目的目的：为了在一个类中装下另一个类。如果不采用泛型，直接一个外部类内含一个内部类实例的话，那么限制太死，这个外部容器只能放下这一种类型的对象。或者，你可以将任何对象转型为Object后放进去，这样的话，我们就丢失了类型信息，得不偿失。而泛型可以解决这个问题。

对于一个具有泛型参数的类而言，需要在类名后添加尖括号写下一个“形参”，然后就像C中的函数一样，在内部使用这个形参代替实际参数化的类使用，比如 `private T t`创建一个 t 名称的T类型的私有域。此外，这个形参还可以用作方法的返回值、方法的参数类型：`T methodA(T arg)` 一个传入T类型、返回T类型的方法。

## 泛型参数的传递和使用

下面展示了泛型的参数设置和泛型类调用的方法：通过在创建泛型类对象引用的时候，声明这个类的泛型参数为一个内部类，那么这个泛型类就可以用于这个内部类对象的装载。`Holder<InnerType> x = new Holder<>();`。

```java
class A {}
class B extends A {}
class C extends B {}
public static void main(String[] args){
    //现在通过初始化，这个泛型类型被指代为A
    Holder2<A> x = new Holder2<>(new C());
    print(x.getElement().getClass());

    //任何指定了泛型类型的类，也可以接受其导出类型的泛型
    x = new Holder2(new B());
    x.getElement();

    //但是，泛型内部的继承不能通过外部的转型
    Holder2<B> y = new Holder2<>(new B()); //构造了一个Holder2<B>
    //Holder2<A> m = y;  // Holder2<B> can't cast to Holder2<A>，因为这里指的是外部的Holder，其
    //没有继承关系
    Holder2<? extends A> z = y;//这样才可以，转型为继承自A的Holder类

    //注意下面这个例子：
    //当不指定Holder泛型参数的时候，T这个指代只是用于内部的逻辑，可以根据外部情况进行变换
    Holder2 p = new Holder2(new B());
    print(p.t.getClass()); //这时候T意味着B？否，因为this.t的类型是B，所以T被看作是B。
    p = new Holder2(new A()); //当下T指的是A，但是这句话一结束，T就什么也不指代了。
    print(p.t.getClass()); //同样，因为t是A，所以T被看作A。
}
```

## 泛型常见的问题

**不使用泛型参数的泛型类会被替换为Object**

如果我们在创建这个泛型类的对象的时候，不使用泛型参数，那么会发生什么？很显然的，这个`<T>`会被替换成Object，使用IDEA等IDE，当你在下列代码调用`p.get...`的时候就会发现参数变成了Object。

**指定参数的泛型引用可以接受这个参数类型及其导出类型的泛型声明**

对于一个指定了泛型参数的泛型类的引用，其可以接受这个泛型参数代表的类及其导出类的泛型参数声明，这没有问题，如下的 `x = new Holder2(new B())` 现在这个 x 表示的是 `Holder2<A>`,因为B是A的导出类型，所以完全正确。注意下面那句 `getElement()` 的返回值，是A。因为x指的是保存A作为参数的泛型，那么这个B自然向上转型为A了。

**泛型类并不会因为泛型参数可以向上转型而自身也可以向上转型**

一个装鱼的篮子和一个装鱼汤的盘子，其中鱼和鱼汤是继承，但是不意味着泛化它们的篮子和盘子之间存在继承。`B是A的导出类不意味着：Holder<B>是Holder<A>的导出类`。因此，声明了B参数的泛型类不能转型为声明了A参数的泛型类，起码采用目前的语法不能（使用？extend可以，见下文）。

## 简单泛型之：元组

如果要让方法返回多个值可能吗？对于C/Java1.4之前的语法，不行，因为我们已经限制过返回值类型了，那里只能写一个参数。但是，如果我们有这样的需求，怎么办？我们需要一个容器，包裹几个不知道什么类型的类的实例，然后能取出来它们。在泛型之前，因为我们不知道需要返回的类型，那么所以没有办法去捕获它们，而又不能设置类型为Object，因为会丢失信息。因此不可行。而泛型用参数的形式捕获了传递进来的类型，因此这些类型得以保存。

泛型内部就像导光条，外部类型就像不同颜色的光，泛型类容器可以传递这些类型而不至于丢失，因此可以用来构造泛型。

```java
import static com.mazhangjing.Print.print;
class TwoTuple<A,B> {
    //这里使用Public并且Final的原因是，我们想要让客户端可以访问，并不能更改（像元组一样）
    public final A first;
    public final B second;
    public TwoTuple(A a,B b){first=a;second=b;}
    public String toString() {
        return "(" + first + ", " + second + ")";
    }
}
class ThreeTuple<A,B,C> extends TwoTuple<A,B> {
    public final C third;
    public ThreeTuple(A a,B b,C c){super(a,b);third=c;}
    public String toString() {
        return "(" + first + ", " + second + ", " + third + ")";
    }
}
class SixTuple<A,B,C,D,E,F> extends ThreeTuple{
    public final D fouth;
    public final E fifth;
    public final F sixth;
    public SixTuple(A a,B b,C c,D d,E e,F f){super(a,b,c);fouth=d;fifth=e;sixth=f;}
    public String toString() {
        return String.format("(%s, %s, %s, %s, %s %s)",first,second,third,fouth,fifth,sixth);
    }
}
print(new TwoTuple<String,String>("Hello","Wolrd"));
print(new ThreeTuple<String,String,Integer>("Hello","Wolrd",23));
(Hello, Wolrd)
(Hello, Wolrd, 23)
```

这个元组目前还需要声明传递进来的参数类型信息，这样才能被正确“导光”，然后保存类型信息。此外，不同长度元组还要使用不同方法，在稍后一些地方，你将看到一个方法适用不同长度参数、不需要声明类型信息的真正元组实现。

## 简单泛型之：嵌套堆栈

下面这个堆栈实现的很有水平。Node保存一个称之为 item 的元素，以及另一个Node。当调用 push 的时候，传入元素，会新建一个 Node 类对象，这个对象保有元素和最初的Node。而再次调用 push 的时候，再次传入元素，再次新建一个 Node 对象，这个 Node 对象将会保存这个元素和上一个Node（而上一个Node则保存了上一个元素和最初的Node）。取出的时候使用 pop，然后取出最外层Node的 item，设置其内含的Node为当前Node，最终取到最初的Node为止。

```java
import static com.mazhangjing.Print.print;
//用来展示使用泛型构造Stack的方法，这个Stack中的Node是一个俄罗斯套娃，包含一个String类和子Node类，当取出时，取出String类，将
//内部的Node类变为外部Node类继续取String，这样就实现了堆栈。
public class LinkedStack<T> {
    private class Node {
        T item;
        Node next;
        Node() { item = null; next = null;}
        Node(T item, Node next){
            this.item = item;
            this.next = next;
        }
        boolean end() { return item == null && next == null; }
    }
    private Node top = new Node();
    public void push(T item){
        top = new Node(item,top);
    }
    public T pop() {
        T result = top.item;
        if (!top.end()){
            top = top.next;
        }
        return result;
    }
    public static void main(String[] args){
        LinkedStack<String> lss = new LinkedStack<>();
        for (String s:"Hello Marvin from China".split(" ")){
            lss.push(s);
        } String s;
        while ((s = lss.pop()) != null) {
            print(s);
        }
    }
}
```

下面的示例展示了对于内部和外部类均采用参数化类型的泛型设计。注意，因为内部类总是要在外部类之后才能初始化和使用，而静态内部类要比外部类更早的使用，因此，对内部类使用参数化类型，之后通过某种机制再让这个类型传递给外部类的参数化类型。

而采用这种方式设计的原因是，我们希望内部的Node是 static 的。因此，参数就要从内部到外部传递 `U->T`。因为 static 方法更早的被加载。这个示例展示了内部类使用参数化类型设计时的问题：初始化顺序问题。

```java
class LinkedStack2<T> {
    private static class Node<U> {
        U item;
        Node<U> next;
        Node() { item = null; next = null;}
        Node(U item, Node<U> next){
            this.item = item;
            this.next = next;
        }
        boolean end() { return item == null && next == null; }
    }
    private Node<T> top = new Node();
    public void push(T item){
        top = new Node(item,top);
    }
    public T pop() {
        T result = top.item;
        if (!top.end()){
            top = top.next;
        }
        return result;
    }
}
```

## 简单容器之：幸运转盘

下面的例子构造了一个用来保存任意元素的容器，然后从这个外部容器中取出内部容器及其类型的方法。

注意，当我们使用 `RandomList x = new RandomList();`,T 被擦除为 Object。在第一次 Set 的时候，T 的类型为 Object 而不是 String ，因为x不是一个设置了参数化类型的泛型类。因此在第二次 set 的时候，可以接受 Int。但是，如果我们使用 `RandomList<String> x = new RandomList<>();` 那么就不能进行第二次 Set，`Error：Set can't applied to Integer`。注意，这里不会发生自动转型的，Java在任何地方都几乎不会自动发生类型转换。只会报错。

```java
import java.util.*;
import static com.mazhangjing.Print.*;

public class RandomList<T> {
    private List<T> list = new ArrayList<>();
    private Random rand = new Random();
    public void set(T element){
        list.add(element);
    }
    public T select(){
        return list.get(rand.nextInt(list.size()));
    }
    public static void main(String[] args){
        RandomList x = new RandomList();
        for (String s : "This is a happy day!".split(" ")){
            x.set(s);
        }
        print(x.select());
        print(x.list);
        
        for (String s : "1 213 321 12 34 12".split(" ")){
            x.set(Integer.parseInt(s));
        }
        print(x.list);
        for (int s = 0; s < 5; s++){
            print(x.select());
            print(x.select().getClass());
        }
    }
}
```

# 泛型接口

**泛型接口参数化类型的作用：传递参数化类型以限定其方法的实现返回特定类型。**

泛型不仅仅可以用于类，还可以用于接口。比如，较为典型的就是生成器，生成器接口使用泛型可以用来表示其生成的内容类型，这样在 `next()` 方法返回的时候就不至于丢失类型，并且能够适用于多个不同类的生成。

可以看到，当我们实现一个接口的时候，需要写清楚实现接口的参数类型。之后，这个接口就像导光条一样设置了 `next()` 返回的类型为 `Person`。这样的话， next() 在接口实现中就会自动返回 Person 对象。一如既往，如果你不写参数类型的话，这里的 `next()` 的返回值，只能是 Object，或者任何你限定的类型（必须在接口定义中限定）。

```java
import java.util.*;
import static com.mazhangjing.Print.*;

interface Generator<T>{ T next();}

class Person {
    public String toString() {
        return this.getClass().getSimpleName();}
}
class Student extends Person {}
class Teacher extends Stuff {}
class Stuff extends Person {}

class PeopleGenerator implements Generator<Person> {
    private static List<Person> list = new ArrayList<>();
    static {
        list.add(new Stuff());
        list.add(new Teacher());
        list.add(new Student());
        list.add(new Student());
    }
    private int ptr = 0;
    public Person next() {
        ptr++;
        if (ptr == list.size()) ptr = 0;
        return list.get(ptr);
    }
}
//调试代码：
PeopleGenerator x = new PeopleGenerator();
for (int i = 0; i < 10; i++){print(x.next());}
```

**高级版本的泛型使用方式是：通过长长的形参链来获取类型信息，然后作为返回值返回**

在上述例子中，我们看到通过实现一个生成器类的方法来特殊化一个生成器，相比较之前的方法，使用泛型几乎没带来什么好处（不使用泛型你也可以限定next的实现返回某种特殊类型）。那么，我们可不可以使用一个通用生成器来只属于某个类的对象呢？

当然可以，只用传递一个这个类的字面量，然后使用“导光条”从这个类的字面量中获取类型信息，在调用 `next()` 的时候这个类的实例即可。下面的 `BasicGenerator` 就是这么做的。可以看到，这里保留了 Generator 的 T 形参，而且设置了 BasicGenerator 的 T 形参，这样的话，通过构造器获取类字面量，就能从 `Class<T>` 中获取到 `T`(导光条)。然后在 `next()` 返回的时候返回这个类型即可。

注意，这里的 static create 方法返回了这个类自身，这样就不用写 new并且手动构造一个 BG 的实例，可以直接用 `next()` 进行生成了。

```java
class BasicGenerator<T> implements Generator<T> {
    private Class<T> type;
    public BasicGenerator(Class<T> type){this.type = type;}
    public T next() {
        try {
            return type.getDeclaredConstructor().newInstance();
        } catch (Exception e) { throw new RuntimeException(e); }
    }
    /**泛型化的方法允许我们不用麻烦的创建：BasicGenerator<MyType>(MyType.class)*/
    public static <T> Generator<T> create(Class<T> type){
        return new BasicGenerator<>(type);
    }
}
class CountedObject {
    private static long counter;
    private final long id = counter++;
    public String toString() {
        return "CountedObject " + id;
    }
}
//调试代码：
//使用create方法可以隐式的构造计数器
Generator<CountedObject> c = BasicGenerator.create(CountedObject.class);
//泛型就像是导光条，光线在这个例子中从CountedObject传输到Generator
//（经过Class<>，BasicGenerator内部以及接口）。
for (int i = 0; i < 5; i++){
    print(c.next());
}
Generator<CountedObject> bc = new BasicGenerator<>(CountedObject.class);
for (int i = 0; i < 5; i++){
    print(bc.next());
}
```

# 泛型方法

## 基础：用于动态设置参数和返回值

泛型类或者泛型接口堆砌使用的目的都很相似：通过“导光条”，延迟获得类型信息，然后对类型信息进行返回或者进一步的处理。但是，我要说，一般情况下，如果需要处理多个类型，那么将一堆泛型参数写在类上难免难以区分（形式化参数一般使用T或者U之类的简写，难以区分）。这就好像接口实现的多重继承一样，不优雅。因此，我们可以对于方法使用泛型，每个方法各自独立，泛型含义不同，这样就进行了多个泛型参数的区分问题。

**当可以使用泛型方法解决的问题，不要上升到泛型类层面。**

对于静态方法，因为其初始化要早于类实例，因此只能使用泛型方法而不能使用泛型类。泛型方法在返回值前使用尖括号表示形参，在参数中写下形参或者形参的某种形式以获取具体的参数类型。

泛型可以和可变参数工作的很好，`T...args` 声明了一串为T参数的参数们。在方法内部，使用T这个形参遍历取出即可。

```java
import java.util.*;
import static com.mazhangjing.Print.print;

public class GenemicMethod {
    //1、基本的泛型方法展示
    public static <T> void whatIs(T element){
        print("=====INFO=====");
        print(element.getClass().getPackageName());
        print(element.getClass().getName());
    }
    public static <T> T getIt(T element){
        return element;
    }
    //2、泛型方法和可变参数工作的很好
    public static <T> List<T> getList(T...args){
        List<T> t = new ArrayList<>();
        for (T e : args){
            t.add(e);
        } return t;
    }
    public static void main(String[] args){
        GenemicMethod.whatIs(new String());
        GenemicMethod.whatIs(13);//3、注意，对于基本类型，会自动转换成为包装器类型
        print(GenemicMethod.getIt("Hello").getClass());

        Map<String,List<Integer>> m = new HashMap<>();
        //4、JavaSE10已经具备了自动类型参数的推断，而在之前版本1.6之前中，需要写：
        //Map<String,List<Integer>> n = new HashMap<String, List<Integer>>();

        print(GenemicMethod.getList(12,213,231,21).toString());
    }
}
=====INFO=====
java.lang
java.lang.String
=====INFO=====
java.lang
java.lang.Integer
class java.lang.String
[12, 213, 231, 21]
```

下面展示了一个使用泛型方法的高级技巧：通过使用上面介绍过的生成器，传递一组空列表，然后对于其中每个元素都迭代 n 次。借助于泛型，可以在一个静态方法中实现这个自动列表填充。

```java
//5、展示了使用静态方法将一个迭代器生成迭代指定次数并且存放到某种容器中并返回
public static <T> Collection<T> fill(Collection<T> c,Generator<T> gen,int n){
    for (int i = 0; i < n; i++){
        c.add(gen.next());
    }
    return c;
}
//重载的版本，不至于丢失类型信息
public static <T> Set<T> fill(Set<T> c, Generator<T> gen, int n){ ... }
public static <T> Queue<T> fill(Queue<T> c, Generator<T> gen, int n){ ... }
//调试代码：
List<Person> p = Ex13.fill(new ArrayList<Person>(), new PeopleGenerator(),3);
print(p);// [Teacher, Student, Student]
```

此外，在上述Generator中，我们使用了一个 create 的静态方法，那里也是一个泛型方法，通过`Class<T>`参数获取到了T，并且传给 `Generator<T>`,这样的话就实现了生成器类型的指定。

```java
public static <T> Generator<T> create(Class<T> type){
    return new BasicGenerator<>(type);
}
```

## 高级示例：通过形参传递实现真正的元组

在之前的元组版本中，我们需要传递对象的类型，对于不同长度的元组要调用不同方法，对于后者，可以使用方法重载解决：设置一个 tuple 的方法，通过接受不同长度的形参，而返回不同长度元素的类来解决。对于前者，当我们通过泛型方法传递形参的时候，自动捕获了这些对象的类型，因此这些类型毫不费力的就传递到了 TwoTuple 中，而这个传递接受的还是形参，因为导光条没断，因此一直到 TwoTuple 内部 `public A a` 调用形参类型创建真实元素的时候才停止形参传递。因此，我们现在可以通过 `import static Tuple2.*` 来使用 tuple 方法: `tuple(12,213,"Hello",'c',MyClass)`，这几乎像Python一样灵活，并且，通过IDE，你也知道其参数的具体类型。

```java
//7、使用泛型方法改进过的元组示例
//总之，泛型有点这样的意思：我不去显式限定你的类型，我只要求你提供它，然后一级一级传下去，一直到
//需要定义的地方再去检查和定义。泛型方法和泛型类、泛型接口都是如此。
class Tuple2 {
    public static <A,B> TwoTuple<A,B> tuple(A a, B b){
        return new TwoTuple<>(a,b);
    }
    public static <A,B,C> ThreeTuple<A,B,C> tuple(A a, B b, C c){
        return new ThreeTuple<>(a,b,c);
    }
    public static <A,B,C,D,E,F> SixTuple<A,B,C,D,E,F> tuple(A a, B b, C c, D d, E e, F f){
        return new SixTuple<>(a,b,c,d,e,f);
    }
    public static void main(String[] args){
        //使用泛型方法简化过的元组就不用写类型了。这就有点动态语言的味道。
        print(Tuple2.tuple("hello","marvin",123));
    }
}
```

## 高级示例：打造通用的集合工具

```java
//8、使用泛型方法创建一个Set实用数学工具
//注意，sets不要和set重名，否则在union中的set会被误以为是类
class Sets {
    public static <T> Set<T> union(Set<T> a, Set<T> b){ //并集
        Set<T> result = new HashSet<T>(a);
        result.addAll(b);
        return result;
    }
    public static <T> Set<T> intersection(Set<T> a, Set<T> b){ //交集
        Set<T> result = new HashSet<>(a);
        result.retainAll(b);
        return result;
    }
    public static <T> Set<T> difference(Set<T> superset, Set<T> subset){ //剔除superset中的subset
        Set<T> result = new HashSet<>(superset);
        result.removeAll(subset);
        return result;
    }
    public static void main(String[] args){
        Set a = new HashSet();
        a.addAll(Arrays.asList(12,13,14,15,16));
        Set b = new HashSet();
        b.addAll(Arrays.asList(12,14,16,18,20));
        print(Sets.union(a,b));
        print(Sets.intersection(a,b));
        print(Sets.difference(a,b));
    }
}
[16, 18, 20, 12, 13, 14, 15]
[16, 12, 14]
[13, 15]
```

## 高级应用：用于参数类型的推断

通过泛型方法写入容器。参见 “逆变” 部分，代码如下：

```java
public static class SuperType2 {
    static <T> void write(List<? super T> list, T item){
        list.add(item);
    }
    public static void main(String[] args){
        List<Fruit> flist = new ArrayList<>();
        List<Apple> alist = new ArrayList<>();
        write(alist,new Apple()); //T现在为Apple
        write(flist,new Apple()); //T现在为Apple
        write(flist,new Orange());//T现在为Orange
        print(flist.get(1).getClass());
    }
}
```

# 匿名内部类中的泛型使用

泛型在匿名内部类中工作很好。可以看到 Generator 这个接口的参数化类型被用作匿名内部类返回，这样的话，我们就可以调用这个静态类来快速填充一些元素了。当然，使用泛化的`BasicGenerator`也可以。这只是个示例。

```java
import java.util.*;
import static com.mazhangjing.Print.print;

public class Ocean {
    public static void main(String[] args){
        LinkedList<SmallFish> line = new LinkedList<>();
        LinkedList<BigFish> line2 = new LinkedList<>();
        GenemicMethod.Ex13.fill(line,SmallFish.generator(),15);
        GenemicMethod.Ex13.fill(line2,BigFish.generator(),15);
        for (SmallFish s : line){print(s);}
        for (BigFish s : line2){print(s);}
    }
}
class SmallFish {
    private static int counter;
    private final int id = counter++;
    public String toString() {
        return "SmallFish " + id;
    }
    public static Generator<SmallFish> generator(){
        return new Generator<SmallFish>() {
            public SmallFish next() {
                return new SmallFish();
            }
        };
    }
}
class BigFish {
    private static int counter;
    private final int id = counter++;
    public String toString() {
        return "BigFish " + id;
    }
    public static Generator<BigFish> generator(){
        return new Generator<BigFish>() {
            public BigFish next() {
                return new BigFish();
            }
        };
    }
}
```

# 泛型编程：更少的类，更少的耦合

没有泛型活不下去吗？当然不是，我们有两种选择，过于泛化的接口/类的继承或者重复写大量代码来实现具体类/接口的结合。

**对于一个接口而言，为什么要限定其泛型参数类型？**

```java
class A implements Comparable {
    @Override public compareTo(Object o){
         //xxxx
    } 
}
class A implements Comparable<A> {
    @Override public compareTo(A o){
         //xxxx
    } 
}
```

由上面可以明显的看出来，A对象实现了一个比较的接口，那么，按照逻辑，A只应该和A类型比较，和别的类型比较无意义，因此需要使用泛型限定类型参数。如果没有泛型，那就太糟糕了，这就好像穿着肥大衣服的瘦子，完全不配套。

在泛型之前，我们解决过于宽泛的问题的思路是：通过继承一个接口：

```java
interface A_Comparable extends Comparable {};
class A implements A_Comparable {...}
```
如果太懒，那么就只能忍受过于松散且混乱的代码。


**对于一个类而言，为什么要限定其泛型参数类型？**

下面以一个容器类为例，探讨为什么我们不使用传统的继承而是要使用泛型参数的限定。这个有点长的模型实际很简单，一个货船，包含了不同的走道，每个走道包含一些货架，每个货架包含一些货物，每个货物有一些属性。

对于Ship而言，其是一个包含有Aisle的List，因此需要 `Ship extend AisleList` 来限定保存 Aisle，那么 AisleList 必须通过 `AisleList extends ShelfList` 来保存货架。而 `ShelfList extends ArrayList` 也是必须的。因此，可以看到一个常常的复杂的继承链条。下面提供了一种使用泛型的方法，泛型限制了ArrayList的元素类型，同时提供了所有的ArrayList代码实现，可以看到，它更加的模块化，更容易解耦。那么，让每个元素都继承没有泛型的ArrayList会怎样呢？太过宽泛，可能发生逻辑上的错误。

下面代码想要说明的是：泛型类用来将类和类联系起来：我们看到了`Ship extends ArrayList<Aisle>`, `Aisle extends ArrayList<Shelf>`,`Shelf extends ArrayList<Goods>`。对于一个ArrayList使用泛型的好处是：对于每个容器都进行了限制，因此每个容器都拥有其内含容器的对象列表，以及对其的访问权限，这样就构造了一个层次化的安全的嵌套容器。


```java
import java.util.ArrayList;
import java.util.Random;
import static com.mazhangjing.Print.print;

public class ShipDemo {
    public static void main(String[] args){
        print(new Ship("Marvin III",10));
    }
}
class Goods {
    private int id;
    private String name;
    private String descripton;
    private String from;
    private String to;
    Goods(int id, String name, String from, String to, String descripton){
        this.id = id; this.from = from; this.to = to;
        this.name = name; this.descripton = descripton;
    }
    public void changeFrom(String from){this.from = from;};
    public void changeTo(String to){this.to = to;}
    public String toString() {
        return String.format("GOODS-%d: %s [%s-%s]",id,name,from,to);
    }
    public static Generator<Goods> generator(){
        Random rand = new Random();
        String[] loc = {"CN","UK","US","HK","TW","JP","IN"};
        return new Generator<Goods>() {
            public Goods next() {
                return new Goods(rand.nextInt(1000),"Foods",
                        loc[rand.nextInt(loc.length)],loc[rand.nextInt(loc.length)],
                        "Foods are good.");
            }
        };
    }
}
class Shelf extends ArrayList<Goods> {
    private int id = new Random().nextInt(100);
    Shelf(int nGoods){
        GenemicMethod.Ex13.fill(this,Goods.generator(),nGoods);
    }
    public String toString() {
        return String.format("SHELF-%d: GOODS TOTAL: %d",id,size());
    }
}
class Aisle extends ArrayList<Shelf> {
    private int id = new Random().nextInt(10);
    Aisle(int nShelfs, int nGoods){
        for (int n = 0; n < nShelfs; n++){
            add(new Shelf(nGoods));
        }
    }
    public String toString() {
        return String.format("AISLE-%d: SHELF TOTAL: %d",id,size());
    }
}
class People {}
class Office {}
class Ship extends ArrayList<Aisle> {
    private String name;
    private Office office = new Office();
    private ArrayList<People> stufflist = new ArrayList<>();
    {
        stufflist.add(new People());
        stufflist.add(new People());
    }
    Ship(String name, int nAisle){
        this.name = name;
        for (int n = 0; n < nAisle; n++){
            add(new Aisle(10,new Random().nextInt(20)+90));
        }
    }
    public String toString() {
        StringBuilder sb = new StringBuilder();
        StringBuilder sb_t = new StringBuilder();
        int g_total = 0 , s_total = 0, a_total = 0;
        for (Aisle a : this){
            sb.append(a);sb.append("\n");
            a_total++;
            for (Shelf s : a){
                sb.append("\t");sb.append(s);sb.append("\n");
                s_total++;
                for (Goods g : s){
                    g_total++;
                }
            }
        }
        sb_t.append(String.format("\n\nSHIP:%s. Aisle NUM: %d. Shelf NUM: %d. Good NUM: %d.\n\n"
                ,name,a_total,s_total,g_total));
        sb_t.append(sb);
        return sb_t.toString();
    }
}
SHIP:Marvin III. Aisle NUM: 10. Shelf NUM: 100. Good NUM: 9670.
AISLE-6: SHELF TOTAL: 10
	SHELF-1: GOODS TOTAL: 100
	SHELF-18: GOODS TOTAL: 100
    ...
...

```

# 擦除

## Java 擦除问题

Java的泛型是后来添加到语言中的，为了能够协调那些并非是通过泛型写的类库以及采用泛型写的新类，Java的泛型通过擦除实现。这意味着：

- 对于指定的类型注解 `<T>`，`List<T>` 会被擦除到 List
- 对于普通的类型变量 T，T 会被擦除到 Object    

这意味着，在编译后，泛型类内部将无法保存任何关于形式参数T的信息。

```java
import java.lang.reflect.Array; import java.util.*;
import static com.mazhangjing.Print.print;

class Tee { void methodA(){} }
public class GyenericType<T> {
    private T t;
    private List<T> list = new ArrayList<>();
    GyenericType(T t) { this.t=t; this.list.add(t); }
    void callSepc(){
        print(Arrays.toString(t.getClass().getTypeParameters()));
        print(Arrays.toString(list.getClass().getTypeParameters()));
        //t.methodA() //错误，擦除意味着不能在内部调用属于类的特殊方法
    }
}
//调用测试：
new GyenericType<Tee>(new Tee()).callSepc(); //[] [E]
```

`getTypeParameters` 返回一个包含了类型声明的数组，可以看到，这里明明声明了Tee作为参数，但实际却是空和 E 。Java 内部从未保存过我们传入的类型声明的参数。当使用泛型的时候，`List<String>`和`List<Integer>`都通过擦除成为`List`，然后进行处理。

## 使用 extends 协变构建边界阻止擦除

对于擦除问题的一个解决办法是，使用 `extends` 关键字说明 T 类型可以接受 Tee 以及其所有导出类，这样的话，擦除就会进行到这个边界：`<T extends Tee> 通过擦除变成 <Tee>`，这样就可以通过在类内部创建一个这个类型的变量，传递一个对象并且使用其方法。

```java
class GyenericType2<T extends Tee> {
    private T t;
    private List<T> list = new ArrayList<>();
    GyenericType2(T t) {this.t=t;this.list.add(t);}
    void callSepc(){
        //使用extends扩展可以塑造边界，但是依然不能访问具体的类型声明
        print(Arrays.toString(t.getClass().getTypeParameters()));
        print(Arrays.toString(list.getClass().getTypeParameters()));
        //边界可以访问属于类的特殊方法了
        t.methodA();
    }
} //调用测试：
new GyenericType2<Tee>(new Tee()).callSepc();
```

因为我们对类使用泛型的目的在于泛化操作，所以在这里调用一个特殊类的方法不合时宜，但是，对于泛化方法而言，这就可能造成问题了：

```java
interface Animal {
    void getName();
    void getInfo();
}
class Cat implements Animal {
    public void getName() {   print("cat");   }
    public void getInfo() {  print("cat.. is cat"); }
    public void getAge() {   print("cat age is short than people.");  }
    public String toString() {  return this.getClass().getSimpleName();  }
}
class House {
    //使用泛型的边界防止擦除而无法使用非接口方法：getAge
    public <T extends Cat> void getCat(T cat){
        cat.getAge();
        cat.getInfo();
        cat.getName();
    }
}//调用测试：
new House().getCat(new Cat());
cat age is short than people. ；  cat.. is cat  ；  cat
```

至于你可能想问，直接在那里写上 void getCat(Cat cat) 不就行了？ 问题是，如果现在有一个 Cat 的导出类，调用的话就会丢失信息（向上转型），而使用泛型方法则不会（按理论来说，对于C++）。

## 擦除并非恶魔：泛型动作依旧发生在边界

虽然有擦除，但是对于对于泛型内部，其亦然具有某种逻辑的一致性。如下，我们接受一个`Class<T>`，然后得到了`T`。使用这个类字面量构造一个类型为`T[]`的数组，显然出错了？没有，这里的`T[]`被擦除到`Object[]`。这意味着，我们无法将其取出了，因为Object丢失了所有的信息。可以查看打印的结果，正是这样。

对于List，结果不太一样，我们依旧使用类字面量构造类对象，添加到List中，而List中保存的虽然还是Object对象，但是打印数组的时候，我们发现得到的竟然是 Cat。Java显然在某个地方保存了这个T的信息，因而对其进行了转型，得到了正常的结果。

根据`javap -c` 查看汇编代码，我们知道，所有在泛型中发生的动作都发生在：

- 对于传递进来的值进行编译期额外的检查
- 对于值要进行传出的地方进行自动转型

而在运行时的其余地方，所有的泛型信息都被擦除了。

```java
class Gyeneric<T> {
    private Class<T> c;
    Gyeneric(Class<T> c) {
        this.c = c;
    }
    @SuppressWarnings("unchecked")
    T[] create(int n){
        //这里使用反射的Array创造对象的数组
        return (T[])Array.newInstance(c,n);
    }
    List<T> createArray(int n){
        List<T> list = new ArrayList<>();
        for (int i = 0; i < n; i++){
            try {
                list.add(c.getDeclaredConstructor().newInstance());
            } catch (Exception e) {throw new RuntimeException(e);}
        }
        return list;
    }
    public static void main(String[] args){
        Gyeneric<Cat> x = new Gyeneric<>(Cat.class);
        print(Arrays.toString(x.create(5)));
        print(x.createArray(5));
    }
}
[null, null, null, null, null]
[Cat, Cat, Cat, Cat, Cat]
```

## 擦除中对象创建问题的补偿措施

由于擦除，造成了 `T[], new T[], (T)new Object[], new T(), instanceof T` 这些方法都不可用。幸好，`isInstance`还可以使用，但是，需要传递一个类字面量进去。

那么，如果我们想要在内部创造一个T的对象呢？下面的 `addType` 和 `createNew` 展示了使用类字面量保存类型信息以及创造类型对象的方法。但是，如果不希望直接创建对象，可以希望使用工厂模式，通过接口方法返回对象，那么使用 `addType2` 保存工厂类，使用 `createNew2` 通过工厂接口方法返回实例可以很好解决这个问题。

```java
import java.util.*; import static com.mazhangjing.Print.*;

interface Fa<T> { T create(); }
class Building {}
class Home extends Building {}
class HomeFactory implements Fa<Home> {
    public Home create(){
        return new Home();
    }
}
public class ClassTypeCapture<T> {
    Class<T> kind;
    Map<String,Class<?>> map = new HashMap<>();
    Map<String,Fa<? extends T>> map2 = new HashMap<>();
    ClassTypeCapture(Class<T> kind){
        this.kind = kind;
    }
    public boolean f(Object arg){
        return kind.isInstance(arg); //少数可用的方法
        //下面展示的几乎任何直接调用T的方式都出错了
        //T[] array = (T)new Object[3]; 错误
        //T[] array = new T[3]; 错误
        //new T(); //错误，原因之一是因为擦除，原因之二是无法验证是否有默认构造器
        //return arg instanceof T;//错误，已被擦除成Object
        //Error:Class or array excepted
    }
    //------------------------类字面量方法示例-----------------------
    public void addType(String typename,Class<?> kind){
        map.put(typename,kind);
    } 
    public Object createNew(String typename){
        Class<?> c = map.get(typename);
        try {
            return c.getDeclaredConstructor().newInstance();
        } catch (Exception e) {throw new RuntimeException(e);}
    }
    //---------------------------工厂方法示例------------------------
    //注意这里的工厂类写法：Factory<? extends T> 保存类型 T 的工厂 Factory
    public void addType2(String typename,Fa<? extends T> factory){
        map2.put(typename,factory);
    }
    public Object createNew2(String typename){
        //成功使用Map在具有泛型的类中创建了一个对象——使用工厂返回一个创建好的对象
        //如果你不想使用类字面量的话，这也是一种解决办法
        return map2.get(typename).create();
    }
} //测试代码：
ClassTypeCapture<Building> x = new ClassTypeCapture<>(Building.class);
print(x.f(new Home())); //true
print(x.f(new Building())); //true

//对于Java而言，如果想要知道确切类型信息，就必须传递类对象以在类型表达式中使用它。
x.addType("HOME",Home.class);
Home h = (Home)x.createNew("HOME");

//如果不想要MAP中存储类字面量，那么也可以存储一个能够返回Home实例的HomeFacroty
x.addType2("HOMEFACTORY",new HomeFactory());
Home h2 = (Home) x.createNew2("HOMEFACTORY");
```

# 通配符

## 协变 extends 

**数组的协变**

定义：

```java
class Fruit {}
class Apple extends Fruit {}
class Orange extends Fruit {}
class Fuji extends Apple {}
```

对于数组而言，我们看到，fruit 是一个Fruit类型的引用，因此其可以添加任何继承自Fruit类的水果，比如橘子、苹果、红富士苹果。这对于编译器不会出错。而我们看到，实际上， fruit 引用指向的数组却是 Apple类型的。因此，当实际我们添加橘子到 fruit时，编译器不会出错，但是错误发生在这个引用找到这个数组的时候。

这里的机制是：第一句话创建了一个苹果数组，并且设置了一个向上转型的引用 fruit，这个向上转型的引用，可以 赋值 ` = new Orange()` ，但是其指向的对象则不接受这种元素，Java在运行时给出了检查。这很好。

```java
Fruit[] fruit = new Apple[4]; //正确，向上转型为Fruit类型的引用
fruit[0] = new Apple(); //正确，对于Fruit类型的引用，可以添加其导出类型
fruit[1] = new Fuji(); //正确，同上
//fruit[2] = new Orange(); //尽管这个数组本身是由Apple创建的，但现在是一个Fruit类型的引用，
//但是当运行时（由引用插入数组本体时）会出错，原因是，运行时知道自己处理的类型是Apple[]。
```

**泛型容器的协变**

但是，对于泛型容器而言，问题就很扯淡了。

第一个问题就是，泛型容器之间不存在继承，因此不能创建一个 `List<Apple>` 然后传递给一个 `List<Fruit>` 的引用。`List<Apple>` 就好像一个装苹果的碗，`List<Fruit>` 就好像一个装水果的麻布袋子，水果和苹果的继承关系不意味着碗和麻布袋子的继承关系很好，但是，这个问题可以解决:通过通配符问号的使用。

这里通配符 `? extends Fruit` 的含义并不是指，任何从Fruit继承的类型的List，而是指一个未知的具体类型，这个类型来自于协变空间：Fruit及其继承类中。因此，这个类型绝对是不安全的，当添加元素到这个类型，根本无从知晓要转换成何种类型，因此就拒绝了添加元素。但是，对于获取元素，则没有安全的问题，可以正常进行。因此我们看到：使用通配符后，Java将不允许添加数据到List，但可以取出extends类型声明的数据。

```java
//List<Fruit> flist = new ArrayList<Apple>();
List<? extends Fruit> flist = new ArrayList<Apple>(Arrays.asList(new Apple()));

flist.add(new Apple()); //错误，原因见下
flist.add(new Fruit()); //错误，原因见下
//flist.add() // 原因是：参数类型为 ? extend Fruit e，编译器无法通过此处安全捕获类型，因此拒绝添加
Fruit f = flist.get(0); //虽然返回类型依然是 ？ extend Fruit，但是可以安全转型为Fruit，因此可行
Apple a = (Apple) flist.get(0);
flist.indexOf(new Apple()); //indexOf接受一个Object参数，安全，因此可行
```

总而言之，因为泛型并非Java天生特性，因此任何通配符协变的List中需要安全确认类型的方法都不可使用：add（基本上意味着不可写入数据）。但是，那些不需要确认，或者能够保证安全的方法可以继续使用：get、indexof、contains等（这些都是取出数据的操作）。

## 逆变 super

逆变和协变正好相反，如下代码所示，逆变采用 `<? super MyClass>` 指的是，这个问号可以接受任何来自于 MyClass 及其基类的类型。这就很好的解决了由于协变造成的类型不安全，无法添加到一个协变引用的问题。

注意，逆变在泛型方法上的应用中，提到了如何利用参数推断将一个元素写入到一个由其基类构成的List中的。

```java
public class SuperType {
    //1、基本逆变：<? super MyClass>
    //使用super Fruit 代替extends，含义为：？是由某个类来界定的，Fruit是其上界
    //如果这里是 super Apple, 那么只有 Fuji 和 Apple 类型可以add
    //这和extends看起来功能相反，extends指的是可以扩充Fruit及其导出类
    public static void main(String[] args){
        List<? extends Fruit> flist = new ArrayList<>();
        //flist.add(); Error, extends Fruit 类型参数不安全，无法解析传入参数具体类型
        List<? super Fruit> flist2 = new ArrayList<>(Arrays.asList(new Apple()));
        flist2.add(new Orange()); //正确，flist2限定为了Fruit，所有添加的元素都被转型为Fruit
        List<? super Apple> alist = new ArrayList<>();
        alist.add(new Fuji());
        //alist.add(new Fruit()); //错误，无法将Fruit转化为Apple类。
    }
    //2、逆变在泛型方法中的应用：<? super T>
    //注意，不同于<T extends MyClass>，不能使用<T super MyClass>!!!!
    public static class SuperType2 {
        static <T> void write(List<? super T> list, T item){
            list.add(item);
        }
        public static void main(String[] args){
            List<Fruit> flist = new ArrayList<>();
            List<Apple> alist = new ArrayList<>();
            write(alist,new Apple()); //T现在为Apple
            write(flist,new Apple()); //T现在为Apple
            write(flist,new Orange());//T现在为Orange
            print(flist.get(1).getClass());
        }
    }
}
```

## 协变和逆变的比较

下面是协变和逆变的一个简单比较。注意，对于逆变而言，因为其接受参数保存更低级的元素，无法安全的向下转型为T，因此返回值为Object。如果要从一个更加泛化的列表中取/放一个特殊元素，使用逆变，反之，使用协变。

```java
class SuperType3 {
    static class Reader<T> {
        T getIt(List<T> item){
            return item.get(0);
        }
        T getIt2(List<? extends T> item){
            return item.get(0);
        }
        Object getIt3(List<? super T> item){
            return item.get(0);
        }//注意，Super than T，那么不能返回T，只能返回Object
    }
    public static void main(String[] args){
        List<Apple> alist = Arrays.asList(new Apple());
        List<Fruit> flist = Arrays.asList(new Apple());
        Reader<Fruit> r = new Reader<>();
        r.getIt(flist);
        //r.getIt(alist); Wrong. Apple can't cast to Fruit
        r.getIt2(alist); // Right. Apple is extends from Fruit

        Reader<Apple> r2 = new Reader<>();
        r2.getIt3(flist); // Right. Fruit is super than Apple.
        r2.getIt3(alist);
    }
}
```

# Java 泛型问题一览

## 泛型不能使用基本类型

泛型不接受基本类型，但是支持自动拆包和封装机制，这可能不算以个问题。

```java
//Holder2<int> 错误
Holder2<Integer> h = new Holder2<>(12);
List<Integer> li = new LinkedList<>();
for (int i = 0; i < 10; i++){
    li.add(i*10);
}
print(li); //0 10 20 30 ...
```

## 不同参数的同一泛型接口冲突

不能对一个类实现同一种泛型接口的两个版本，因为存在擦除，所以这两个实现会变得冲突。当使用基本的带有参数的接口并且继承别的类也实现了这个接口的时候，很容易由于擦除出现这个问题。

```java
interface If1<T> {}
class My1 implements If1 {}
class My2 extends My1 implements If1 {}
//如果不写泛型，那么可以正常使用，因为这个接口在My2中即是继承又是实现

class My3 implements If1<My3> {}
//extends My3 就默认具有了 If1<My3> 的My3的接口。
class My4 extends My3 implements If1<My1> {} //错误，冲突
```

## 重载方法不可根据泛型参数区分

```java
class WWW<U,V>{
    void f(List<U> u){}
    //void f(List<V> v){} 由于擦除，造成了冲突
    void f2(List<V> v){}
}
```

## 基类劫持了接口的泛型参数类型

这个非常容易出现，因为Pets最好和Pets比较，那么在基类就使用了参数的泛型接口，但是，对于子类而言，也想之和子类比较，结果发现，因为继承过来了一个Pets的泛型参数类型，那么，就没有办法再实现一个Cats的泛型参数类型。当然，解决办法是有的，多写点代码就是了。

```java
class Pets implements Comparable<Pets>{
    public int compareTo(Pets o) {
        return 0;
    }
}
//class Cats extends Pets implements Comparable<Cats> {
class Cats extends Pets {
    @Override public int compareTo(Pets p){
        if (this.getClass().isInstance(p)) {
            //dosomething
        }
        return 0;
    }//这个不是重载的版本，只是方法的名字很像罢了
    public int compareTo(Cats o) {
        return 0;
    }
}
```

这还是因为擦除的锅！！

## 猫容器中的狗

将狗通过某个接受List的方法添加到猫的容器中。

因为一个容器中保存的对象是无法直接区分类型的，全是Object，然后需要转型才能引用其中的元素。因此，通过addIt方法，而不是直接调用那个绝对是猫的cats引用，就可以将狗添加到猫群中。

```java
class Safe {
    static void addIt(List list){
        list.add(new Dogs());
    }
    public static void main(String[] args){
        List<Cats> cats2 = new LinkedList<>();
        //cats2.add(new Dogs()); 无法添加，cats2是一个猫的泛型容器引用，
        //但是，如果使用不通过这个引用呢？
        addIt(cats2); //很容易就将一只狗添加到猫群里了
        print(cats2); //Fucking dog!!!!!

        List<Cats> cats = new LinkedList<>();
        cats = Collections.checkedList(cats,Cats.class);
        //cats.add(new Dogs());
        //addIt(cats); //WOW!无法添加，这会出错
    }
}
class Dogs {
    public String toString() {
        return "I'm a dog!";
    }
}
```
事实证明，这种状况很少出现。

# 动态类型安全

在上面例子中，解决办法就是，通过将一个列表通过 checkList 进行检查后，再去添加，这样就不会出现问题。这里可能是一个动态代理实现的。总而言之，也算是Java容器的一个类型安全解决方案。

# 潜在类型机制的补偿

C++ 中和Python中有一种叫做潜在类型机制的玩意儿，俗称鸭子类型。如果一个东西看起来像鸭子，叫起来像鸭子，那么就把它当作鸭子处理。区别是，C++在编译期进行检查，而Python则是在运行时抛出错误。Java由于擦除没有这种特性，但是，可以通过以下方式弥补。

1、接口

通过实现一个包含了看起来和听起来的接口，通过泛型方法传递一个协变的泛型，就间接的做到了潜在类型机制能够做到的事情。当然，这样和直接采用接口没有任何区别。

```java
interface MayBe_Duck {
    void speak();
    void sit();
}
class Duck1 implements MayBe_Duck{
    public void speak() {

    }
    public void sit() {

    }
}
class Act {
    <T extends MayBe_Duck> void maybe_duck (T t){
        t.sit();
        t.speak();
    }
    void maybe_duck2 (MayBe_Duck t){
        t.speak();
        t.sit();
    }
}
```

2、反射

此外，使用反射（虽然稍微有点慢），也可以很好解决这个问题。

```java
void maybe_duck3 (Object animal){
    try {
        Method s = animal.getClass().getMethod("speak");
        s.invoke(animal);
        s = animal.getClass().getMethod("sit");
        s.invoke(animal);
    } catch (Exception e) {throw new RuntimeException(e);}
}
```